In [1]:
# This notebook have built from this tutorial: https://github.com/bnsreenu/python_for_microscopists/blob/master/330_Detectron2_Instance_3D_EM_Platelet.ipynb
import os

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

## Load Dataset

In [2]:
from src.segmentation.framework_handlers.detectron2_handler import register_and_split_dataset

segments_dataset_name = "etaylor/stigmas_dataset"
release = "v0.2"

train_metadata, train_dataset_dicts, test_metadata, test_dataset_dicts = register_and_split_dataset(
    dataset_name=segments_dataset_name,
    release_version=release,
    train_ratio=0.75,  # 75% for training, 25% for testing
)

print(f"Train dataset: {len(train_dataset_dicts)} samples")
print(f"Test dataset: {len(test_dataset_dicts)} samples")


Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 115/115 [00:00<00:00, 547.21it/s]


Initialized dataset with 115 images.
Exporting dataset. This may take a while...


100%|██████████| 115/115 [01:17<00:00,  1.48it/s]


Exported to ./export_coco-instance_etaylor_stigmas_dataset_v0.2.json. Images in segments/etaylor_stigmas_dataset/v0.2
WARNING [02/23 20:13:11 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/23 20:13:11 d2.data.datasets.coco]: Loaded 115 images in COCO format from segments/etaylor_stigmas_dataset/annotations/export_coco-instance_etaylor_stigmas_dataset_v0.2.json
Train dataset: 86 samples
Test dataset: 29 samples


In [4]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
import os
from datetime import datetime
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo
from detectron2.checkpoint import DetectionCheckpointer
import torch
from detectron2.engine import AMPTrainer
import gc


# Define paths and parameters
detectron2_models_path = "/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/detectron2/fine_tuned"
num_epochs = 50
batch_size = 2
learning_rate = 0.00025
num_classes = 1

# Dictionary of models to train
models = {
    "mask_rcnn_R_50_FPN_3x": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",
    "mask_rcnn_R_101_FPN_3x": "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml",
    "mask_rcnn_X_101_32x8d_FPN_3x": "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml",
    "mask_rcnn_R_50_C4_3x": "COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml",
    "mask_rcnn_R_50_DC5_3x": "COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x.yaml"
}

# Function to train a model
def train_model(model_name, config_path):
    # Create configuration
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_path))
    cfg.DATASETS.TRAIN = (f"{segments_dataset_name}_train",)
    cfg.DATASETS.TEST = (f"{segments_dataset_name}_test",)
    cfg.INPUT.MASK_FORMAT = "bitmask"
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_path)  # Initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = batch_size
    cfg.SOLVER.BASE_LR = learning_rate
    cfg.SOLVER.MAX_ITER = 3000
    cfg.SOLVER.STEPS = []  # Do not decay learning rate
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # Default is 512
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes  # Set number of classes
    cfg.SOLVER.AMP.ENABLED = True

    # Set output directory
    model_saving_path = os.path.join(detectron2_models_path, model_name)
    cfg.OUTPUT_DIR = model_saving_path
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    # Train the model
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # Save checkpoint after training
    checkpointer = DetectionCheckpointer(trainer.model, save_dir=cfg.OUTPUT_DIR)
    checkpointer.save("model_final")

# Train each model
for model_name, config_path in models.items():
    print(f"Training {model_name}...")
    train_model(model_name, config_path)
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Finished training {model_name}. Model saved in {os.path.join(detectron2_models_path, model_name)}")


In [8]:
import os
import gc
import json
import torch
from datetime import datetime
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Define paths and parameters
DETECTRON2_MODELS_PATH = "/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/detectron2/fine_tuned"
NUM_EPOCHS = 50
BATCH_SIZE = 4
LEARNING_RATE = 0.00025
NUM_CLASSES = 1

# Dataset names
TRAIN_DATASET = f"{segments_dataset_name}_train"
TEST_DATASET = f"{segments_dataset_name}_test"

# Dictionary of models to train
MODELS = {
    "mask_rcnn_R_50_FPN_3x": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",
    "mask_rcnn_R_101_FPN_3x": "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml",
    "mask_rcnn_X_101_32x8d_FPN_3x": "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml",
    "mask_rcnn_R_50_C4_3x": "COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml",
    "mask_rcnn_R_50_DC5_3x": "COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x.yaml"
}


def train_and_evaluate_model(model_name, config_path):
    """Train and evaluate a single model."""
    print(f"Processing {model_name}...")
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_path))
    cfg.DATASETS.TRAIN = (TRAIN_DATASET,)
    cfg.DATASETS.TEST = (TEST_DATASET,)
    cfg.INPUT.MASK_FORMAT = "bitmask"
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_path)
    cfg.SOLVER.IMS_PER_BATCH = BATCH_SIZE
    cfg.SOLVER.BASE_LR = LEARNING_RATE
    cfg.SOLVER.MAX_ITER = 1000
    cfg.SOLVER.STEPS = []  # No learning rate decay
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
    cfg.SOLVER.AMP.ENABLED = True
    
    # Set output directory
    cfg.OUTPUT_DIR = os.path.join(DETECTRON2_MODELS_PATH, model_name)
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    # Train the model
    print(f"Training {model_name}...")
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    # Save final model checkpoint
    checkpointer = DetectionCheckpointer(trainer.model, save_dir=cfg.OUTPUT_DIR)
    checkpointer.save("model_final")
    
    # Free up memory
    gc.collect()
    torch.cuda.empty_cache()
    
    print(f"Finished training {model_name}. Model saved in {cfg.OUTPUT_DIR}")
    
    # Evaluate the model
    print(f"Evaluating {model_name}...")
    checkpointer.load(os.path.join(cfg.OUTPUT_DIR, "model_final.pth"))
    evaluator = COCOEvaluator(TEST_DATASET, output_dir=cfg.OUTPUT_DIR, max_dets_per_image=1000, use_fast_impl=True)
    val_loader = build_detection_test_loader(cfg, TEST_DATASET)
    eval_results = inference_on_dataset(trainer.model, val_loader, evaluator)
    
    # Save evaluation results
    output_dir = os.path.join(cfg.OUTPUT_DIR, "evaluation")
    os.makedirs(output_dir, exist_ok=True)
    results_file = os.path.join(output_dir, "eval_results.json")
    
    with open(results_file, "w") as f:
        json.dump(eval_results, f, indent=4)
    
    print(f"Evaluation completed. Results saved in {results_file}")


In [9]:
for model_name, config_path in MODELS.items():
    train_and_evaluate_model(model_name, config_path)

Processing mask_rcnn_R_50_FPN_3x...
Training mask_rcnn_R_50_FPN_3x...
[02/22 15:41:47 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): Froz

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[02/22 15:41:51 d2.engine.train_loop]: Starting training from iteration 0


/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/detectron2/data/detection_utils.py:446: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/

[02/22 15:42:10 d2.utils.events]:  eta: 0:07:58  iter: 19  total_loss: 2.491  loss_cls: 0.797  loss_box_reg: 0.3043  loss_mask: 0.6945  loss_rpn_cls: 0.6238  loss_rpn_loc: 0.08114    time: 0.6995  last_time: 0.3670  data_time: 0.3788  last_data_time: 0.0049   lr: 4.9953e-06  max_mem: 2664M
[02/22 15:42:24 d2.utils.events]:  eta: 0:08:20  iter: 39  total_loss: 2.399  loss_cls: 0.6748  loss_box_reg: 0.2898  loss_mask: 0.6878  loss_rpn_cls: 0.6079  loss_rpn_loc: 0.08068    time: 0.6972  last_time: 0.5955  data_time: 0.3452  last_data_time: 0.2705   lr: 9.9902e-06  max_mem: 2896M
[02/22 15:42:39 d2.utils.events]:  eta: 0:08:42  iter: 59  total_loss: 1.953  loss_cls: 0.4928  loss_box_reg: 0.3004  loss_mask: 0.6763  loss_rpn_cls: 0.3411  loss_rpn_loc: 0.08559    time: 0.7060  last_time: 0.8765  data_time: 0.3724  last_data_time: 0.5401   lr: 1.4985e-05  max_mem: 2896M
[02/22 15:42:54 d2.utils.events]:  eta: 0:08:12  iter: 79  total_loss: 1.866  loss_cls: 0.4208  loss_box_reg: 0.439  loss_mas

...

[02/22 16:31:07 d2.utils.events]:  eta: 0:14:59  iter: 19  total_loss: 2.624  loss_cls: 0.6655  loss_box_reg: 0.3797  loss_mask: 0.6941  loss_rpn_cls: 0.7077  loss_rpn_loc: 0.09745    time: 0.9861  last_time: 0.8883  data_time: 0.1255  last_data_time: 0.0069   lr: 4.9953e-06  max_mem: 5853M
[02/22 16:31:27 d2.utils.events]:  eta: 0:15:03  iter: 39  total_loss: 2.769  loss_cls: 0.5889  loss_box_reg: 0.4151  loss_mask: 0.6873  loss_rpn_cls: 0.8453  loss_rpn_loc: 0.1543    time: 0.9809  last_time: 1.0870  data_time: 0.0575  last_data_time: 0.1312   lr: 9.9902e-06  max_mem: 5853M
[02/22 16:31:48 d2.utils.events]:  eta: 0:15:04  iter: 59  total_loss: 2.332  loss_cls: 0.492  loss_box_reg: 0.464  loss_mask: 0.6759  loss_rpn_cls: 0.595  loss_rpn_loc: 0.1279    time: 1.0004  last_time: 1.2276  data_time: 0.0512  last_data_time: 0.0047   lr: 1.4985e-05  max_mem: 5853M
[02/22 16:32:12 d2.utils.events]:  eta: 0:15:14  iter: 79  total_loss: 2.043  loss_cls: 0.4178  loss_box_reg: 0.4769  loss_mask: 

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[02/22 16:48:36 d2.engine.train_loop]: Starting training from iteration 0


/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/detectron2/data/detection_utils.py:446: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be supp

[02/22 16:48:56 d2.utils.events]:  eta: 0:13:48  iter: 19  total_loss: 4.084  loss_cls: 0.7874  loss_box_reg: 0.2366  loss_mask: 0.6961  loss_rpn_cls: 2.016  loss_rpn_loc: 0.1529    time: 0.9491  last_time: 0.8947  data_time: 0.2318  last_data_time: 0.0035   lr: 4.9953e-06  max_mem: 6082M
[02/22 16:49:13 d2.utils.events]:  eta: 0:13:31  iter: 39  total_loss: 3.023  loss_cls: 0.6483  loss_box_reg: 0.3048  loss_mask: 0.6895  loss_rpn_cls: 1.148  loss_rpn_loc: 0.1157    time: 0.8884  last_time: 0.8570  data_time: 0.0355  last_data_time: 0.0053   lr: 9.9902e-06  max_mem: 6082M
[02/22 16:49:31 d2.utils.events]:  eta: 0:13:19  iter: 59  total_loss: 2.589  loss_cls: 0.5202  loss_box_reg: 0.4703  loss_mask: 0.6749  loss_rpn_cls: 0.7492  loss_rpn_loc: 0.1144    time: 0.8919  last_time: 0.8550  data_time: 0.0659  last_data_time: 0.0602   lr: 1.4985e-05  max_mem: 6196M
[02/22 16:49:50 d2.utils.events]:  eta: 0:13:14  iter: 79  total_loss: 2.291  loss_cls: 0.4537  loss_box_reg: 0.5557  loss_mask: 